# Training using cross-validation on RecTools model

Here, I train and evaluate diffrent models on each split, using [RecTools models and metrics implementation](https://rectools.readthedocs.io/en/stable/features.html).

## Data reading

In [97]:
N_USERS = 943
N_ITEMS = 1682
K = 10
data_interim_dir = '../data/interim/'
data_filenames = [f'u{t}.{split}' for t in ['1', '2', '3', '4', '5', 'a', 'b'] for split in ['base', 'test']]
data_filenames

['u1.base',
 'u1.test',
 'u2.base',
 'u2.test',
 'u3.base',
 'u3.test',
 'u4.base',
 'u4.test',
 'u5.base',
 'u5.test',
 'ua.base',
 'ua.test',
 'ub.base',
 'ub.test']

In [98]:
import pickle


data = {}
for i in range(0, len(data_filenames), 2):
    base_filename, test_filename = data_filenames[i:i+2]
    data_title = base_filename.split('.')[0]
    with open(data_interim_dir + base_filename + '.pickle', 'rb') as base:
        with open(data_interim_dir + test_filename + '.pickle', 'rb') as test:
            with open(data_interim_dir + test_filename + '.df.pickle', 'rb') as base_df:
                with open(data_interim_dir + test_filename + '.df.pickle', 'rb') as test_df:
                    data[data_title] = (pickle.load(base), pickle.load(test), pickle.load(base_df), pickle.load(test_df))

data.keys()

dict_keys(['u1', 'u2', 'u3', 'u4', 'u5', 'ua', 'ub'])

## Performing 5-fold cross-validation on `u1-u5` splits

In [105]:
def load_data(fold):
    return data[f'u{fold}']

In [106]:
from rectools import Columns
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd


def train_and_validate(model, fold_data):
    train, test, train_df, test_df = fold_data

    model.fit(train)
    
    recos = model.recommend(
        users=train_df[Columns.User].unique(),
        dataset=train,
        k=K,
        filter_viewed=True,
    )
    recos.rename(columns={Columns.Score: Columns.Weight}, inplace=True)

    merged_data = pd.merge(recos, test_df, on=[Columns.User, Columns.Item], suffixes=('_predicted', '_test'))
    rmse = np.sqrt(mean_squared_error(merged_data[Columns.Weight + '_test'], merged_data[Columns.Weight + '_predicted']))
    
    return rmse

In [153]:
def cross_validation(model, model_name, num_folds=5):
    print(model_name)
    
    rmse_values = []
    for fold in range(1, num_folds + 1):
        fold_data = load_data(fold)
        fold_rmse = train_and_validate(model, fold_data)
        rmse_values.append(fold_rmse)
        print(f"RMSE (Fold {fold}): {fold_rmse}")

    average_rmse = np.mean(rmse_values)
    print(f"Average RMSE (across all folds): {average_rmse}\n")

In [154]:
from lightfm import LightFM
from rectools.models import PureSVDModel, LightFMWrapperModel, ImplicitALSWrapperModel
from implicit.als import AlternatingLeastSquares


factors = 10  # Fine-tuned
model_svd = PureSVDModel()
model_als = ImplicitALSWrapperModel(
        AlternatingLeastSquares(factors=factors)
        )
model_lfm = LightFMWrapperModel(
    model=LightFM(no_components=factors, k=K),
    epochs=1,  # Fine-tuned
    )

In [155]:
cross_validation(model_svd, 'PureSVDModel')

PureSVDModel
RMSE (Fold 1): 2.465804801558191
RMSE (Fold 2): 2.1509809413755785
RMSE (Fold 3): 2.0662233143866886
RMSE (Fold 4): 2.1081675292333175
RMSE (Fold 5): 2.152631851093425
Average RMSE (across all folds): 2.18876168752944



In [156]:
cross_validation(model_als, 'AlternatingLeastSquares')

AlternatingLeastSquares


c:\ProgramData\anaconda3\envs\py310\lib\site-packages\rectools\dataset\features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

RMSE (Fold 1): 2.5380366492540736


...

RMSE (Fold 4): 2.3608011215578353


c:\ProgramData\anaconda3\envs\py310\lib\site-packages\rectools\dataset\features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

RMSE (Fold 5): 2.367598727982355
Average RMSE (across all folds): 2.4088144648013894



In [157]:
cross_validation(model_lfm, 'LightFM')

LightFM
RMSE (Fold 1): 28.933907612732494
RMSE (Fold 2): 28.994876857271887
RMSE (Fold 3): 29.60348985984716
RMSE (Fold 4): 29.170035175190396
RMSE (Fold 5): 28.775662757717225
Average RMSE (across all folds): 29.09559445255183



### Choosing the best model

In [158]:
best_model = model_svd
best_model_name = 'PureSVD'

## Test on A and B splits

In [159]:
from rectools import Columns
from rectools.metrics import NDCG, Accuracy, MAP, MCC, MRR


ndcg = NDCG(k=K, log_base=3)
acc = Accuracy(k=K)
mmap = MAP(k=K)
mcc = MCC(k=K)
mrr = MRR(k=K)

def calculate_metrics(recos, fold_data):
    train, test, train_df, test_df = fold_data
    metrics = {
        'MAP':  mmap.calc(reco=recos, interactions=test_df),
        'Accuracy': acc.calc(reco=recos, interactions=test_df, catalog=train_df[Columns.Item]),
        'NDCG': ndcg.calc(reco=recos, interactions=test_df),
        'MCC': mcc.calc(reco=recos, interactions=test_df, catalog=train_df[Columns.Item]),
        'MRR': mrr.calc(reco=recos, interactions=test_df),
    }
    return metrics

def train_and_evaluate(model, fold_data):
    train, test, train_df, test_df = fold_data
    model.fit(train)
    recos = model.recommend(
        users=train_df[Columns.User].unique(),
        dataset=train,
        k=K,
        filter_viewed=True,
    )
    recos.rename(columns={Columns.Score: Columns.Weight}, inplace=True)
    return calculate_metrics(recos, fold_data)

In [162]:
from tabulate import tabulate


def print_metrics_table(metrics_dict):
    table = []

    for metric_name, metric_value in metrics_dict.items():
        table.append([metric_name, metric_value])

    print(tabulate(table, headers=['Metric', 'Value'], tablefmt='simple'))


def test(model, folds=['a', 'b']):
    total_metrics = {
        'MAP': 0,
        'Accuracy': 0,
        'NDCG': 0,
        'MCC': 0,
        'MRR': 0,
    }
    for fold in folds:
        fold_data = load_data(fold)
        fold_metrics = train_and_evaluate(model, fold_data)
        for metric_name, metric_value in fold_metrics.items():
            total_metrics[metric_name] += metric_value
        print(f"Fold {fold}:")
        print_metrics_table(fold_metrics)
        print()
    average_metrics = {metric_name: total_value / len(folds) for metric_name, total_value in total_metrics.items()}
    print(f"Average across test folds:")
    print_metrics_table(average_metrics)

In [163]:
test(best_model)

Fold a:
Metric       Value
--------  --------
MAP       0.149509
Accuracy  0.998394
NDCG      0.285174
MCC       0.242038
MRR       0.613322

Fold b:
Metric       Value
--------  --------
MAP       0.141989
Accuracy  0.99837
NDCG      0.273182
MCC       0.230786
MRR       0.592713

Average across test folds:
Metric       Value
--------  --------
MAP       0.145749
Accuracy  0.998382
NDCG      0.279178
MCC       0.236412
MRR       0.603017
